In [2]:
text = "This is some text"
byte_ary = bytearray(text,"utf-8")
print(bytearray)

<class 'bytearray'>


In [3]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


In [4]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


#####  BPE tokenizers have a vocabulary where we have a token ID for whole words or subwords instead of each character

### BPE Algorithm outline
### 1. Identify frequent pairs

In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters)

### 2. Replace and record

Replace that pair with a new placeholder ID (one not already in use, e.g., if we start with 0...255, the first placeholder would be 256)
Record this mapping in a lookup table
The size of the lookup table is a hyperparameter, also called "vocabulary size" (for GPT-2, that's 50,257)

### 3. Repeat until no gains

Keep repeating steps 1 and 2, continually merging the most frequent pairs
Stop when no further compression is possible (e.g., no pair occurs more than once)
Decompression (decoding)

To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table

In [5]:
# implementation will start here